In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import csv
import sys
import os
from scipy import stats
import math
from scipy.optimize import curve_fit
from scipy.stats import norm
import seaborn as sns
import pandas as pd
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

import powerlaw
from natsort import natsorted

In [3]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [4]:
def load_pd_positions(dirPath, experiment_type):
    if experiment_type != "experiment" and experiment_type != "baseline":
        print("experiment_type could be only $experiment or $baseline")
        exit(-1)

    num_experiment = len([name for name in os.listdir(dirPath) if
                          (os.path.isfile(os.path.join(dirPath, name)) and (name.endswith('position.tsv')))])

    if os.path.exists(dirPath + "/" + experiment_type + ".pkl"):
        return num_experiment, pd.read_pickle(dirPath + "/" + experiment_type + ".pkl")
    # else:
    #     print("Baseline:"+dirPath+" not an existing path")
    #     exit(-1)

    print("Generating pickle positions file in " + dirPath + "/" + experiment_type + ".pkl")
    df = pd.DataFrame()
    for filename in os.listdir(dirPath):
        if filename.endswith('position.tsv'):
            if not os.path.getsize(os.path.join(dirPath, filename)) > 0:
                print("Error, empty file at:" + os.path.join(dirPath, filename))
                continue
            df_single = pd.read_csv(dirPath + "/" + filename, sep="\t")
            df = df.append(df_single)

    df.to_pickle(dirPath + "/" + experiment_type + ".pkl")
    return num_experiment, df

In [5]:
def distance_from_the_origin(positions):
    distances = np.array([])
    # df_values1 = df_values[:3,:4]
    for i in range(positions.shape[1]):
        #     print(i)
        x_pos = positions[:, i, 0]
        y_pos = positions[:, i, 1]
        distances = np.vstack([distances, np.sqrt(x_pos ** 2 + y_pos ** 2)]) if distances.size else np.sqrt(
            x_pos ** 2 + y_pos ** 2)
    # print(distances)
    # print("distances shape", distances.shape)
    return distances
        

In [ ]:
df_experiment

In [ ]:
# for i in positions_concatenated:
#     print(i, end='\n\n\n')

In [6]:
def get_connections(positions):
    connection_number_history = np.zeros((positions.shape[0],positions.shape[1]))
    for index,pos_t in enumerate(positions.transpose(1, 0, 2)):
        connection_number = np.zeros(positions.shape[0])
    #     print(c.shape)
        for idx, elem in enumerate(pos_t):
    #         print('\t',elem)
            robot_distance = np.sqrt(np.sum((pos_t-elem)**2, axis=1))
    #         print(robot_distance, end='\n\n\n')
            connection_number[idx] = robot_distance[np.where(robot_distance <= 0.1)].size - 1
        connection_number_history[:,index] = connection_number
    return connection_number_history

In [7]:
def get_mean_connection_number_history(folder):
    [number_of_experiments, df_experiment] = load_pd_positions(folder, "experiment")

    positions_concatenated = df_experiment.values[:,1:] #10 robots, 4 time_step
    [num_robot, num_times] = positions_concatenated.shape
    positions_concatenated = np.array([x.split(',') for x in positions_concatenated.ravel()],dtype=float)
    positions_concatenated = positions_concatenated.reshape(num_robot,num_times,2)

    position_concatenated_split = np.split(positions_concatenated, 50)

    connection_number_history = np.array([])
    for single_run in position_concatenated_split:
        connection_number_history = np.vstack((connection_number_history, get_connections(single_run))) if connection_number_history.size else get_connections(single_run)
        
    return np.mean(connection_number_history, axis=0)

In [ ]:
# print(positions_concatenated.shape)

In [ ]:
# print(len(position_concatenated_split[0][0]))

In [ ]:
# print(connection_number_history.shape)

In [ ]:
folder_experiments = "/home/luigi/Documents/ARGoS/argos_results_scripts/test/bouncing_angle_50"
dir_list = os.listdir(folder_experiments)
# dir_list.sort()
print(dir_list)

In [ ]:
for dirName in natsorted(os.listdir(folder_experiments)):
    
    print(os.path.join(folder_experiments,dirName))

In [ ]:
# folder_experiments = "/home/luigi/Documents/ARGoS/argos_results_scripts/test/bouncing_angle_50"

# n_robots = [10, 20, 50, 100]


# Ncolors = len(n_robots)+1
# colormap = plt.cm.viridis  # LinearSegmentedColormap
# Ncolors = min(colormap.N, Ncolors)
# mapcolors = [colormap(int(x * colormap.N / Ncolors)) for x in range(Ncolors)]


# runs = 50
# alpha_array = [1.2, 1.6, 2.0]
# rho_array = [0.0, 0.3, 0.6, 0.9]


# for a in alpha_array:
#     for r in rho_array:
# #         print("a",a,"r",r)
#         robot_array = []
#         fig = plt.figure(figsize=(20, 10), dpi=80)
        
        
        
#         for dirName, subdirList, fileList in os.walk(folder_experiments):
#             num_robots = "-1"
#             elements = dirName.split("_")
#             for e in elements:
#                 if e.startswith("robots"):
#                     num_robots = e.split("#")[-1]
#                 if (e.startswith("rho")):
#                     rho = float(e.split("#")[-1])
#                 if (e.startswith("alpha")):
#                     alpha = float(e.split("#")[-1])

#             #         print(num_robots+' '+str(rho)+' '+str(alpha))
# #             print(alpha, rho)
#             if (num_robots == "-1" or rho != r or alpha != a):
#                 continue
#             else:
# #                 print(dirName)
                
#                 robot_array += [int(num_robots)]
#                 print(num_robots, int(num_robots))
#                 runs = len([f for f in fileList if f.endswith('position.tsv')])
#         #         print(runs)

#             rho_str = str(rho)
#             alpha_str = str(alpha)
            
#             [_, df_experiment] = load_pd_positions(dirName, "experiment")

#             positions_concatenated = df_experiment.values[:,1:]
#             [num_robot, num_times] = positions_concatenated.shape
#             positions_concatenated = np.array([x.split(',') for x in positions_concatenated.ravel()],dtype=float)
#             positions_concatenated = positions_concatenated.reshape(num_robot,num_times,2)

#             position_concatenated_split = np.split(positions_concatenated, runs)

#             connection_number_history = np.array([])
#             for single_run in position_concatenated_split:
#                 connection_number_history = np.vstack((connection_number_history, get_connections(single_run))) if connection_number_history.size else get_connections(single_run)

#             connection_number_history_mean = np.mean(connection_number_history, axis=0)
# #             print(num_times)
# #             print(connection_number_history_mean.shape)
#             print(len(robot_array))
#             plt.plot(np.arange(num_times), connection_number_history_mean, linewidth=2, label=num_robots, color=mapcolors[len(robot_array)])
           
#         plt.title("Average connection with " + r" $\bf{\rho}:$" + rho_str + " and " + r"$\bf{\alpha}:$" + alpha_str)
#         plt.ylabel('mean connection link')
#         plt.xlabel('time')
#         #     plt.legend(loc='lower right')
#         plt.yticks(np.arange(0, 9, 0.5))
#         #     plt.grid(which='minor')
#         plt.grid()
#         plt.legend()
#         file_name = "average_connection_plot_robots_%s_rho_%s_alpha_%s.png" % (num_robots, rho_str, alpha_str)
#         plt.savefig("connection_plot/"+file_name)
#         plt.close(fig)
# #         plt.show()

In [21]:
folder_experiments = "/home/luigi/Documents/ARGoS/argos_results_scripts/test/bouncing_angle_50"

for dirName in natsorted(os.listdir(folder_experiments)):
    dirPath = os.path.join(folder_experiments,dirName)
#     os.path.isfile(os.path.join(DIR, name))
    
#     for file in os.listdir(os.path.join(folder_experiments,dirName)):
#     print(dirPath)
    
    runs = len([f for f in os.listdir(dirPath) if (os.path.isfile(os.path.join(dirPath, f)) and f.endswith('position.tsv'))])
#     print(runs)

# Average connection plots among different population sizes

In [22]:
folder_experiments = "/home/luigi/Documents/ARGoS/argos_results_scripts/test/bouncing_angle_50"

n_robots = [10, 20, 50, 100]


Ncolors = len(n_robots)+1
colormap = plt.cm.viridis  # LinearSegmentedColormap
Ncolors = min(colormap.N, Ncolors)
mapcolors = [colormap(int(x * colormap.N / Ncolors)) for x in range(Ncolors)]


alpha_array = [1.2, 1.6, 2.0]
rho_array = [0.0, 0.3, 0.6, 0.9]


for a in alpha_array:
    for r in rho_array:
#         print("a",a,"r",r)
        robot_array = []
        fig = plt.figure(figsize=(20, 10), dpi=80)
        
        
        
#         for dirName, subdirList, fileList in os.walk(folder_experiments):
        for dirName in natsorted(os.listdir(folder_experiments)):
            dirPath = os.path.join(folder_experiments,dirName)
            
            num_robots = "-1"
            elements = dirName.split("_")
            for e in elements:
                if e.startswith("robots"):
                    num_robots = e.split("#")[-1]
                if (e.startswith("rho")):
                    rho = float(e.split("#")[-1])
                if (e.startswith("alpha")):
                    alpha = float(e.split("#")[-1])

            #         print(num_robots+' '+str(rho)+' '+str(alpha))
#             print(alpha, rho)
            if (num_robots == "-1" or rho != r or alpha != a):
                continue
            else:
#                 print(dirName)
                
                robot_array += [int(num_robots)]
#                 print(num_robots, int(num_robots))
                runs = len([f for f in os.listdir(dirPath) if (os.path.isfile(os.path.join(dirPath, f)) and f.endswith('position.tsv'))])
        #         print(runs)

            rho_str = str(rho)
            alpha_str = str(alpha)
            
            [_, df_experiment] = load_pd_positions(dirPath, "experiment")

            positions_concatenated = df_experiment.values[:,1:]
            [num_robot, num_times] = positions_concatenated.shape
            positions_concatenated = np.array([x.split(',') for x in positions_concatenated.ravel()],dtype=float)
            positions_concatenated = positions_concatenated.reshape(num_robot,num_times,2)

            position_concatenated_split = np.split(positions_concatenated, runs)

            connection_number_history = np.array([])
            for single_run in position_concatenated_split:
                connection_number_history = np.vstack((connection_number_history, get_connections(single_run))) if connection_number_history.size else get_connections(single_run)

            connection_number_history_mean = np.mean(connection_number_history, axis=0)
#             print(num_times)
#             print(connection_number_history_mean.shape)
#             print(len(robot_array))
            plt.plot(np.arange(num_times), connection_number_history_mean, linewidth=2, label=num_robots, color=mapcolors[len(robot_array)])
           
        
        plt.title("Average connection with " + r" $\bf{\rho}:$" + rho_str + " and " + r"$\bf{\alpha}:$" + alpha_str)
        plt.ylabel('mean connection link')
        plt.xlabel('time')
        #     plt.legend(loc='lower right')
        plt.yticks(np.arange(0, 9, 0.5))
        #     plt.grid(which='minor')
        plt.grid()
        plt.legend(loc=2)#, bbox_to_anchor=(0.95, 0.5))
        file_name = "average_connection_plot_robots_%s_rho_%s_alpha_%s.png" % (num_robots, rho_str, alpha_str)
        plt.savefig("connection_plot/"+file_name)
        plt.close(fig)
#         plt.show()

In [ ]:
# # mean_grad = np.mean(connection_number_history, axis=0)
# n_robots = [10, 20, 50, 100]
# fig = plt.figure(figsize=(20, 10), dpi=80)


# Ncolors = len(n_robots)
# colormap = plt.cm.viridis  # LinearSegmentedColormap
# Ncolors = min(colormap.N, Ncolors)
# mapcolors = [colormap(int(x * colormap.N / Ncolors)) for x in range(Ncolors)]


# for i,n_r in enumerate(n_robots):
#     folder = "/home/luigi/Documents/ARGoS/argos_results_scripts/test/"+str(n_r)
# #     connection_number_history_experiment.append(get_mean_connection_number_history(folder))
#     plt.plot(times, get_mean_connection_number_history(folder), linewidth=2, label=str(n_r), color=mapcolors[i])

# plt.title("Mean connection link")
# plt.ylabel('mean connection link')
# plt.xlabel('time')
# #     plt.legend(loc='lower right')
# plt.yticks(np.arange(0, 7, 0.5))
# #     plt.grid(which='minor')
# plt.grid()
# plt.legend()
# plt.show()


# # folder_20 = "/home/luigi/Documents/ARGoS/argos_results_scripts/test/20"
# # connection_number_history_experiment.append(get_mean_connection_number_history(folder_20))
# # folder_50 = "/home/luigi/Documents/ARGoS/argos_results_scripts/test/50"
# # connection_number_history_experiment.append(get_mean_connection_number_history(folder_50))
# # folder_100 = "/home/luigi/Documents/ARGoS/argos_results_scripts/test/100"
# # connection_number_history_experiment.append(get_mean_connection_number_history(folder_100))
# # times = np.arange(mean_grad.size)


Qui hai le **posizioni**

In [ ]:
folder = "/home/luigi/Documents/ARGoS/argos_results_scripts/test/100"
[number_of_experiments, df_experiment] = load_pd_positions(folder, "experiment")

positions_concatenated = df_experiment.values[:,1:] #10 robots, 4 time_step
[num_robot, num_times] = positions_concatenated.shape
positions_concatenated = np.array([x.split(',') for x in positions_concatenated.ravel()],dtype=float)
positions_concatenated = positions_concatenated.reshape(num_robot,num_times,2)

position_concatenated_split = np.split(positions_concatenated, 50)

print(positions_concatenated.shape)

Qui hai il **numero di connessioni**

In [ ]:
connection_number_history = np.array([])
for single_run in position_concatenated_split:
    connection_number_history = np.vstack((connection_number_history, get_connections(single_run))) if connection_number_history.size else get_connections(single_run)
    
print(connection_number_history.shape)

Qui hai  le **distanze** dallorigine

In [ ]:
# distance_from_the_origin(position_concatenated_split[0])

In [ ]:
distance_0 = distance_from_the_origin(positions_concatenated).T
# print(distance_0.min())

In [ ]:
bin_edges = np.linspace(0, 0.45, 20)
print(bin_edges)

In [ ]:
connection_t0 = np.ones((bin_edges.size-1, distance_0.shape[1])) * -1

for idx,distance_t in enumerate(distance_0.T):
#     print(distance_t)
    for edge_idx in range(bin_edges.size-1):
#         print(bin_edges[edge_idx],bin_edges[edge_idx+1])
#         print("\t",edge_idx)
        where_index = np.where(np.logical_and(distance_t>= bin_edges[edge_idx], distance_t< bin_edges[edge_idx+1]))
        connection_t0[edge_idx,idx] = np.mean(connection_number_history[where_index])

In [ ]:
# print(connection_t0.shape)
# print(connection_t0)

In [ ]:

print(bin_edges)

In [ ]:
print(np.mean(connection_number_history[where_index]))

In [ ]:
prova = np.empty(3)

In [ ]:
prova

In [ ]:
hist, bins = np.histogram(distance_0, bin_edges)

In [ ]:
hist

In [ ]:
bins

In [23]:
def connection_heatmap(edges, time_connections, alpha, rho, num_robots):
    y_edges = edges[1:].round(decimals=3)
    fig = plt.figure(figsize=(10, 5), dpi=160)
#     plt.ylabel('distance from the origin')
#     plt.xlabel('time(s)')
#     plt.legend()
    yticks = y_edges
    ax = sns.heatmap(time_connections, yticklabels=yticks, cmap="viridis")
    ax.set_title(
        "Average connection from the origin with " + r"$\bf{Robots}$:" + num_robots + r" $\bf{\rho}:$" + rho + " and " + r"$\bf{\alpha}:$" + alpha)
    file_name = "average_connection_heatmap_robots_%s_rho_%s_alpha_%s.png" % (num_robots, rho, alpha)
    plt.savefig("connection_plot/"+file_name)
    plt.close(fig)
#     ax.set_title('Average connection from the origin')
#     plt.show()

In [24]:
folder_experiments = "/home/luigi/Documents/ARGoS/argos_results_scripts/test/bouncing_angle_50"
# runs = 50
bin_edges = np.linspace(0, 0.45, 20)
# print(bin_edges)

for dirName, subdirList, fileList in os.walk(folder_experiments):
    num_robots = "-1"
    rho = -1.0
    alpha = -1.0
    elements = dirName.split("_")
    for e in elements:
        if e.startswith("robots"):
            num_robots = e.split("#")[-1]
        if (e.startswith("rho")):
            rho = float(e.split("#")[-1])
        if (e.startswith("alpha")):
            alpha = float(e.split("#")[-1])

    #         print(num_robots+' '+str(rho)+' '+str(alpha))
    if (num_robots == "-1" or rho == -1.0 or alpha == -1):
        continue
    else:
        print(dirName)
        runs = len([f for f in fileList if f.endswith('position.tsv')])
#         print(runs)
        
    rho_str = str(rho)
    alpha_str = str(alpha)
    
    #POSIZIONI
    [_, df_experiment] = load_pd_positions(dirName, "experiment")
    
    positions_concatenated = df_experiment.values[:,1:] #10 robots, 4 time_step
    [num_robot, num_times] = positions_concatenated.shape
    positions_concatenated = np.array([x.split(',') for x in positions_concatenated.ravel()],dtype=float)
    positions_concatenated = positions_concatenated.reshape(num_robot,num_times,2)

    position_concatenated_split = np.split(positions_concatenated, runs)

    print("positions_concatenated.shape: ",positions_concatenated.shape)
    
    #CONNESSIONI
    connection_number_history = np.array([])
    for single_run in position_concatenated_split:
        connection_number_history = np.vstack((connection_number_history, get_connections(single_run))) if connection_number_history.size else get_connections(single_run)

    print(connection_number_history.shape)
    
    #DISTANZE DALL'ORIGINE
    origin_distance = distance_from_the_origin(positions_concatenated).T
    
    connection_in_time = np.ones((bin_edges.size-1, origin_distance.shape[1])) * -1

    for idx,distance_t in enumerate(origin_distance.T):
    #     print(distance_t)
        for edge_idx in range(bin_edges.size-1):
    #         print(bin_edges[edge_idx],bin_edges[edge_idx+1])
    #         print("\t",edge_idx)
            where_index = np.where(np.logical_and(distance_t>= bin_edges[edge_idx], distance_t< bin_edges[edge_idx+1]))
            connection_in_time[edge_idx,idx] = np.mean(connection_number_history[where_index])

    connection_heatmap(bin_edges, connection_in_time, alpha_str, rho_str, num_robots)

/home/luigi/Documents/ARGoS/argos_results_scripts/test/bouncing_angle_50/2020-06-13_robots#20_alpha#1.6_rho#0.9_baseline_1800
positions_concatenated.shape:  (1000, 181, 2)
(1000, 181)


/home/luigi/.virtualenvs/argos/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/luigi/.virtualenvs/argos/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


/home/luigi/Documents/ARGoS/argos_results_scripts/test/bouncing_angle_50/2020-06-13_robots#50_alpha#1.2_rho#0.6_baseline_1800
positions_concatenated.shape:  (2500, 181, 2)
(2500, 181)
/home/luigi/Documents/ARGoS/argos_results_scripts/test/bouncing_angle_50/2020-06-13_robots#10_alpha#1.2_rho#0.9_baseline_1800
positions_concatenated.shape:  (500, 181, 2)
(500, 181)


/home/luigi/.virtualenvs/argos/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/luigi/.virtualenvs/argos/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


/home/luigi/Documents/ARGoS/argos_results_scripts/test/bouncing_angle_50/2020-06-13_robots#100_alpha#1.6_rho#0.6_baseline_1800
positions_concatenated.shape:  (5000, 181, 2)
(5000, 181)
/home/luigi/Documents/ARGoS/argos_results_scripts/test/bouncing_angle_50/2020-06-13_robots#10_alpha#2.0_rho#0.9_baseline_1800
positions_concatenated.shape:  (500, 181, 2)
(500, 181)


...

/home/luigi/.virtualenvs/argos/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/luigi/.virtualenvs/argos/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


/home/luigi/Documents/ARGoS/argos_results_scripts/test/bouncing_angle_50/2020-06-13_robots#50_alpha#1.6_rho#0.6_baseline_1800
positions_concatenated.shape:  (2500, 181, 2)
(2500, 181)
/home/luigi/Documents/ARGoS/argos_results_scripts/test/bouncing_angle_50/2020-06-13_robots#100_alpha#1.2_rho#0.0_baseline_1800
positions_concatenated.shape:  (5000, 181, 2)
(5000, 181)
/home/luigi/Documents/ARGoS/argos_results_scripts/test/bouncing_angle_50/2020-06-13_robots#10_alpha#1.2_rho#0.0_baseline_1800
positions_concatenated.shape:  (500, 181, 2)
(500, 181)


/home/luigi/.virtualenvs/argos/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/luigi/.virtualenvs/argos/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


/home/luigi/Documents/ARGoS/argos_results_scripts/test/bouncing_angle_50/2020-06-13_robots#50_alpha#1.2_rho#0.0_baseline_1800
positions_concatenated.shape:  (2500, 181, 2)
(2500, 181)
/home/luigi/Documents/ARGoS/argos_results_scripts/test/bouncing_angle_50/2020-06-13_robots#10_alpha#1.2_rho#0.3_baseline_1800
positions_concatenated.shape:  (500, 181, 2)
(500, 181)


/home/luigi/.virtualenvs/argos/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/luigi/.virtualenvs/argos/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
# print(mean_grad.shape)
# print(times.shape)

In [ ]:
# fig = plt.figure(figsize=(20, 10), dpi=80)
# plt.title("10 robs")
# plt.ylim(0, 4)
# plt.plot(times, mean_grad, linewidth=2)


In [ ]:
# fig = plt.figure(figsize=(20, 10), dpi=80)
# plt.title("20 robs")
# plt.ylim(0, 4)
# plt.plot(times, mean_grad, color='green', linewidth=2)


In [ ]:
# fig = plt.figure(figsize=(20, 10), dpi=80)
# plt.title("50 robs")
# plt.ylim(0, 6)
# plt.plot(times, mean_grad, color='blue', linewidth=2)

In [ ]:
# fig = plt.figure(figsize=(20, 10), dpi=80)
# plt.title("100 robs")
# # plt.ylim(0, 6)
# plt.plot(times, mean_grad, color='gray', linewidth=2)

In [ ]:
def distance_from_the_origin(df_values):
    distances = np.array([])
    # df_values1 = df_values[:3,:4]
    for i in range(df_values.shape[1]):
        #     print(i)
        x_pos = df_values[:, i, 0]
        y_pos = df_values[:, i, 1]
        distances = np.vstack([distances, np.sqrt(x_pos ** 2 + y_pos ** 2)]) if distances.size else np.sqrt(
            x_pos ** 2 + y_pos ** 2)
    # print(distances)
    # print("distances shape", distances.shape)
    return distances

In [ ]:
distances = distance_from_the_origin(positions_concatenated)

In [ ]:
distances.T

In [ ]:
distances

In [ ]:
fig = plt.figure(figsize=(50, 15), dpi=80)
ax = sns.heatmap(connection_number_history, cmap="viridis")

# 20

In [ ]:
folder = "/home/luigi/Documents/ARGoS/argos_results_scripts/test/20"
[number_of_experiments, df_experiment] = load_pd_positions(folder, "experiment")

positions_concatenated = df_experiment.values[:,1:] #50 robots
[num_robot, num_times] = positions_concatenated.shape
positions_concatenated = np.array([x.split(',') for x in positions_concatenated.ravel()],dtype=float)
positions_concatenated = positions_concatenated.reshape(num_robot,num_times,2)

print(positions_concatenated.shape)

In [ ]:
connection_number_history = np.zeros((positions_concatenated.shape[0],positions_concatenated.shape[1]))
for index,col in enumerate(positions_concatenated.transpose(1, 0, 2)):
    connection_number = np.zeros(positions_concatenated.shape[0])
#     print(c.shape)
    for idx, elem in enumerate(col):
#         print('\t',elem)
        robot_distance = np.sqrt(np.sum((col-elem)**2, axis=1))
#         print(robot_distance, end='\n\n\n')
        connection_number[idx] = robot_distance[np.where(robot_distance <= 0.07)].size - 1
    connection_number_history[:,index] = connection_number

In [ ]:
fig = plt.figure(figsize=(50, 15), dpi=80)
ax = sns.heatmap(connection_number_history, cmap="viridis")

# 50

In [ ]:
folder = "/home/luigi/Documents/ARGoS/argos_results_scripts/test/50"
[number_of_experiments, df_experiment] = load_pd_positions(folder, "experiment")

positions_concatenated = df_experiment.values[:,1:] #50 robots
[num_robot, num_times] = positions_concatenated.shape
positions_concatenated = np.array([x.split(',') for x in positions_concatenated.ravel()],dtype=float)
positions_concatenated = positions_concatenated.reshape(num_robot,num_times,2)

print(positions_concatenated.shape)

In [ ]:
connection_number_history = np.zeros((positions_concatenated.shape[0],positions_concatenated.shape[1]))
for index,col in enumerate(positions_concatenated.transpose(1, 0, 2)):
    connection_number = np.zeros(positions_concatenated.shape[0])
#     print(c.shape)
    for idx, elem in enumerate(col):
#         print('\t',elem)
        robot_distance = np.sqrt(np.sum((col-elem)**2, axis=1))
#         print(robot_distance, end='\n\n\n')
        connection_number[idx] = robot_distance[np.where(robot_distance <= 0.07)].size - 1
    connection_number_history[:,index] = connection_number

In [ ]:
fig = plt.figure(figsize=(50, 15), dpi=80)
ax = sns.heatmap(connection_number_history, cmap="viridis")

# 100

In [ ]:
folder = "/home/luigi/Documents/ARGoS/argos_results_scripts/test/100"
[number_of_experiments, df_experiment] = load_pd_positions(folder, "experiment")

positions_concatenated = df_experiment.values[:,1:] #50 robots
[num_robot, num_times] = positions_concatenated.shape
positions_concatenated = np.array([x.split(',') for x in positions_concatenated.ravel()],dtype=float)
positions_concatenated = positions_concatenated.reshape(num_robot,num_times,2)

print(positions_concatenated.shape)

In [ ]:
connection_number_history = np.zeros((positions_concatenated.shape[0],positions_concatenated.shape[1]))
for index,col in enumerate(positions_concatenated.transpose(1, 0, 2)):
    connection_number = np.zeros(positions_concatenated.shape[0])
#     print(c.shape)
    for idx, elem in enumerate(col):
#         print('\t',elem)
        robot_distance = np.sqrt(np.sum((col-elem)**2, axis=1))
#         print(robot_distance, end='\n\n\n')
        connection_number[idx] = robot_distance[np.where(robot_distance <= 0.07)].size - 1
    connection_number_history[:,index] = connection_number

In [ ]:
fig = plt.figure(figsize=(50, 15), dpi=80)
ax = sns.heatmap(connection_number_history, cmap="viridis")

In [ ]:
fig = plt.figure(figsize=(50, 15), dpi=80)
ax = sns.heatmap(connection_number_history, cmap="viridis")

In [ ]:
first_elem = positions_concatenated[0,0,:]

In [ ]:
first_col + first_elem

In [ ]:
robot_distance = np.sqrt(np.sum((first_col-first_col[0])**2, axis=1))
print(robot_distance)
robot_in_range = robot_distance[np.where(robot_distance <= 0.07)].size - 1

In [ ]:
robot_in_range

In [ ]:
a = np.arange(20) * 2
a = a.reshape(10,2)

In [ ]:
a

In [ ]:
(a-a[1])**2

In [ ]:
np.sum((a-a[1])**2, axis=1)

In [ ]:
comm_range = np.sqrt(np.sum((a-a[1])**2, axis=1))
print(comm_range)

In [ ]:
comm_range[np.where(comm_range <= 10)].size - 1

In [ ]:
np.where(comm_range <= 10)

In [ ]:
dist = np.linalg.norm(a-b, axis = 0)

In [ ]:
dist

In [ ]:
np.ones(6) * 2